In [ ]:
# Lab | Imbalanced data - Solutions

In [61]:
# Instructions - Import Libraries
import imblearn #need to istall in jupyter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [62]:
# Load the dataset and explore the variables.
data = pd.read_csv('/Users/nferrari/Documents/IronHack/Git-repo/lab-imbalanced-data/files_for_lab/customer_churn.csv')
data.head()

#churn is the variable of customers who stop subscription of being clients

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [63]:
# Explore the three variables of interest
data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [64]:
# We will try to predict variable Churn using a logistic regression on variables tenure, SeniorCitizen,MonthlyCharges.

In [65]:
# Extract the target variable.
y = data['Churn']
y

0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7038     No
7039     No
7040     No
7041    Yes
7042     No
Name: Churn, Length: 7043, dtype: object

In [66]:
#transform y into dummy
y = pd.get_dummies(y,drop_first=True)
y.head()

,Yes
0,0
1,0
2,1
3,0
4,1


In [67]:
# Extract the independent variables and scale them.
x_num = data.select_dtypes(include = np.number)

In [68]:
# Scale them but it should have asked to split test first, no?
transformer = StandardScaler().fit(x_num)
X_scaled = pd.DataFrame(transformer.transform(x_num),columns=x_num.columns)
X_scaled.head()

,SeniorCitizen,tenure,MonthlyCharges
0,-0.439916,-1.277445,-1.160323
1,-0.439916,0.066327,-0.259629
2,-0.439916,-1.236724,-0.362660
3,-0.439916,0.514251,-0.746535
4,-0.439916,-1.236724,0.197365


In [69]:
# Build the logistic regression model.
from sklearn.linear_model import LogisticRegression

In [70]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=0)

In [71]:
LR = LogisticRegression(random_state=0, solver='lbfgs') #random for results being reproducable
LR.fit(X_train, y_train)
LR.score(X_test, y_test)

/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7808063600227144

In [72]:
print(y_test.value_counts())

#there is an substantial imbalance in the actual data:
# NO = 3 * YES

Yes
0      1298
1       463
dtype: int64


In [73]:
# this is prove of how the above mentioned disparity influenced the predictions towards the most recurrent category NO

pd.Series(predictions).value_counts()

No     1737
Yes     376
dtype: int64

In [74]:
# to investigate further I ran a correlation matrix:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

pred = LR.predict(X_test)

print("precision: ",precision_score(y_test,pred)) 
#out of all the people who i said that they have a subscription, i was right 61% of the time

print("recall: ",recall_score(y_test,pred)) 
# out of all the people who actually have subscription, i correctly say to 45% of them have a subscription.
# In my prediction, I am losing 55% of the people who i said they DO NOT have a subscrition but they have it.

print("f1: ",f1_score(y_test,pred))

precision:  0.6115942028985507
recall:  0.4557235421166307
f1:  0.5222772277227723


In [75]:
confusion_matrix(y_test,pred)

array([[1164,  134],
       [ 252,  211]])

In [ ]:
# Even a simple model will give us more than 70% accuracy. Why?

## # Evaluate the model

This model clearly highlight a strong prediction coefficient (80%). However, by checking the precision and recall coefficients taken by the confusion matrix, instead of the accuracy of the model, it is clearly possible to see how this specific model is not a good fit for the data and inquery we have asked (see above in the section where precision and recall are explained).

## # Even a simple model will give us more than 70% accuracy. Why?
This problem arise when our depedent variable is not very well distributed. Logistic regression have a tendency to favor the classes with the highest amount of value and the target variable 'Churn' fits exacly this issue. this is clearly visibile above, when I visualized the value count for the test and predicted data of Y, showing the higher amount of NO(s) over YES(s).

In [ ]:
# Synthetic Minority Oversampling TEchnique (SMOTE) is an over sampling technique based on nearest neighbors that adds new points between existing points. Apply imblearn.over_sampling.SMOTE to the dataset. Build and evaluate the logistic regression model. Is it there any improvement?

In [76]:
from imblearn.over_sampling import SMOTE

In [79]:
sm = SMOTE(random_state=100,sampling_strategy='minority',k_neighbors=3)
X_train_SMOTE,y_train_SMOTE = sm.fit_resample(X_train,y_train)

In [80]:
LR = LogisticRegression(max_iter=1000)
LR.fit(X_train_SMOTE, y_train_SMOTE)
pred = LR.predict(X_test)

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

precision:  0.4776978417266187
recall:  0.7170626349892009
f1:  0.5734024179620035


/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [81]:
LR = LogisticRegression(random_state=0, solver='lbfgs') #random for results being reproducable
LR.fit(X_train_SMOTE, y_train_SMOTE)
LR.score(X_test, y_test)

/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.7194775695627484

## Improved Model
As expected, we are seeing a clear improvement in the recall coefficient as the distribution of the depedent variable has been normalized. 

It is interesting to notice that the overall fit of the model decrease.

Precision also decrease due to the fact that the data, being normalized, made it more difficult to specifically assert predictions in a specific direction